In [ ]:
import tensorflow as tf

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
from tensorflow.keras.datasets import mnist
(x_train, labels_train), (x_test, labels_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(labels_train, 10)
y_test = to_categorical(labels_test, 10)

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [ ]:
import tensorflow as tf
from keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, Input
from tensorflow.keras.layers import BatchNormalization
from keras.models import Sequential
model = Sequential([
    layers.Input(shape=(28, 28, 1)),
    Conv2D(32, kernel_size=3, activation="relu", input_shape=(28, 28, 1)),
    BatchNormalization(),
    Conv2D(32, kernel_size=3, activation="relu"),
    BatchNormalization(),
    Conv2D(32, kernel_size=5, activation="relu", strides=2, padding="same"),
    BatchNormalization(),
    Dropout(0.4),
    
    Conv2D(32, kernel_size=3, activation="relu"),
    BatchNormalization(),
    Conv2D(32, kernel_size=3, activation="relu"),
    BatchNormalization(),
    Conv2D(64, kernel_size=5, activation="relu", strides=2, padding="same"),
    BatchNormalization(),
    Dropout(0.4),
    Conv2D(64, kernel_size=5, activation="relu", strides=2, padding="same"),
    BatchNormalization(),
    Dropout(0.4),
    Conv2D(64, kernel_size=5, activation="relu", strides=2, padding="same"),
    BatchNormalization(),
    Dropout(0.4),
    Conv2D(64, kernel_size=5, activation="relu", strides=2, padding="same"),
    BatchNormalization(),
    Dropout(0.4),
    Conv2D(64, kernel_size=5, activation="relu", strides=2, padding="same"),
    BatchNormalization(),
    Dropout(0.4),
    Conv2D(128, kernel_size=5, activation="relu", strides=2, padding="same"),
    BatchNormalization(),
    Dropout(0.4),
    Conv2D(128, kernel_size=5, activation="relu", strides=2, padding="same"),
    BatchNormalization(),
    Dropout(0.4),
    Flatten(),
    Dense(256, activation="relu"),
    BatchNormalization(),
    Dropout(0.4),
    Dense(10, activation="softmax"),
])



In [ ]:
# Generate new data
from keras.callbacks import LearningRateScheduler
# specify the arguments
rotation_range_val = 10
width_shift_val = 0.1
height_shift_val = 0.1
shear_range_val= 10
zoom_range_val=0.1

# import relevant library
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# create the class object
datagen = ImageDataGenerator(rotation_range = rotation_range_val,
                             width_shift_range = width_shift_val,
                             height_shift_range = height_shift_val,
                             zoom_range=zoom_range_val)

# fit the generator
datagen.fit(x_test.reshape(x_test.shape[0], 28, 28, 1))

# Total number of test data
num = 40000

# Generate new augmented data
for new_x_test, y in datagen.flow(x_test.reshape(x_test.shape[0], 28, 28, 1),labels_test.reshape(labels_test.shape[0], 1),batch_size=num,shuffle=False):
    break
    
new_labels_test = y.flatten()
lr_schedule = LearningRateScheduler(lambda epoch: 1e-3 * (0.95**epoch))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
history = model.fit(x_train, y_train,validation_data=(x_test, y_test),callbacks=[lr_schedule],epochs=20,batch_size=128)

Epoch 1/20
469/469 [==============================] - 359s 753ms/step - loss: 1.6916 - accuracy: 0.4032 - val_loss: 2.9585 - val_accuracy: 0.2735 - lr: 0.0010
Epoch 2/20
469/469 [==============================] - 329s 703ms/step - loss: 0.5338 - accuracy: 0.8276 - val_loss: 0.1249 - val_accuracy: 0.9698 - lr: 9.5000e-04
Epoch 3/20
469/469 [==============================] - 335s 714ms/step - loss: 0.2351 - accuracy: 0.9423 - val_loss: 0.0759 - val_accuracy: 0.9834 - lr: 9.0250e-04
Epoch 4/20
469/469 [==============================] - 331s 707ms/step - loss: 0.1679 - accuracy: 0.9612 - val_loss: 0.0714 - val_accuracy: 0.9855 - lr: 8.5737e-04
Epoch 5/20
469/469 [==============================] - 334s 712ms/step - loss: 0.1334 - accuracy: 0.9697 - val_loss: 0.0558 - val_accuracy: 0.9880 - lr: 8.1451e-04
Epoch 6/20
469/469 [==============================] - 328s 700ms/step - loss: 0.1235 - accuracy: 0.9728 - val_loss: 0.0635 - val_accuracy: 0.9873 - lr: 7.7378e-04
Epoch 7/20
469/469 [======

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')

plt.legend()

In [ ]:
import numpy as np
outputs=model.predict(x_test)
labels_predicted=np.argmax(outputs, axis=1)
misclassified=sum(labels_predicted!=labels_test)
misclassified_percentage = print('Percentage misclassified = ',100*misclassified/labels_test.size)
print('Accuracy = ',100-(100*misclassified/labels_test.size))

313/313 [==============================] - 9s 13ms/step
Percentage misclassified =  0.39
Accuracy =  99.61
